<a href="https://colab.research.google.com/github/wkdwlgus/ktcloud_genai/blob/main/20251015_%EC%9E%84%EB%B2%A0%EB%94%A9_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF_%E1%84%91%E1%85%A1%E1%84%8B%E1%85%B5%E1%86%AB%E1%84%90%E1%85%B2%E1%84%82%E1%85%B5%E1%86%BC_%E1%84%8C%E1%85%A1%E1%86%BC%E1%84%8C%E1%85%B5%E1%84%92%E1%85%A7%E1%86%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **임베딩 모델 미세 튜닝하기**

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

In [ ]:
%%capture
# datasets를 MTEB 호환 버전 설치
!pip install 'datasets>=2.19.0,<3.0.0'

# pydantic을 MTEB 호환 버전 설치
!pip install 'pydantic>=2.0,<2.6'

# MTEB 최신 버전 설치
!pip install 'mteb>=1.39.0'



---



## **미세 튜닝(Fine-tuning) 이해**



### **미세 튜닝(Fine-tuning)이란**?

- **정의** : 이미 방대한 데이터로 학습된 **사전 학습 모델**(Pre-trained Model)을 가져와, 필요한 특정 작업(Task)에 맞게 소량의 데이터로 추가 학습시켜 성능을 최적화하는 과정

### **왜 미세 튜닝을 할까?**

- **시간과 비용 절약**:
    - 이미 만들어진 모델을 활용하므로 부담을 획기적으로 줄여줄 수 있다.
- **높은 성능 달성**:
    - 사전 학습 모델은 이미 언어의 문법, 문맥, 의미 등 방대한 지식을 갖추고 있어 이 지식을 바탕으로 특정 작업에 맞게 약간만 조정해도 매우 높은 성능을 낼 수 있다.
- **데이터 부족 문제 해결**:
    - 특정 작업을 위한 대규모 데이터셋을 구축하기는 어렵다. 미세 튜닝은 상대적으로 적은 양의 데이터만으로도 모델을 효과적으로 학습시킬 수 있게 해준다.

### **미세 튜닝 과정**

1. **사전 학습 모델 선택**:
    - 해결하려는 작업과 가장 **관련성이 높은 범용 모델**(예: BERT, GPT, KoELECTRA 등)을 선택

2. **데이터셋 준비**:
    - 풀고 싶은 특정 작업에 맞는 **소규모의 레이블링된 데이터셋을 준비** (예: 영화 리뷰 감성 분류 데이터, 법률 문서 개체명 인식 데이터)

3. **모델 구조 변경**:
    - 사전 학습 모델의 맨 마지막 단에 풀고 싶은 작업에 맞는 **새로운 층(Layer)을 추가**. 예를 들어, 문장 분류를 하고 싶다면 분류를 위한 '분류 헤드(Classification Head)'를 추가한다.

4. **추가 학습**:
    - 준비된 데이터셋으로 모델을 추가 학습시킨다. 이때, 기존의 지식이 손상되지 않도록 **낮은 학습률**(learning rate)을 사용하는 것이 매우 중요하다. 이는 기존 지식은 유지하면서 새로운 작업에 맞게 가중치를 '미세하게' 조정하는 효과를 준다.

5. **평가**:
    - 학습된 모델이 해당 작업에서 얼마나 좋은 성능을 보이는지 평가한다.

### **사전 학습 vs. 미세 튜닝**

|구분	|사전 학습 (Pre-training)	|미세 튜닝(Fine-tuning)|
|---|---|---|
|**목표**|	언어 자체에 대한 범용적인 이해 능력 확보|	특정 작업(분류, 요약 등)에 대한 전문성 확보|
|**데이터**|	대규모의 레이블 없는 텍스트 (e.g., 웹 전체)|	소규모의 레이블 있는 특정 도메인 텍스트|
|**학습 규모**|	수 주 ~ 수 개월, 수백 대의 GPU 필요|	수 분 ~ 수 시간, 소수의 GPU로 가능|
|**결과물**|	범용 언어 모델 (Foundation Model)|	특정 작업에 특화된 모델 (Specialized Model)|

### **미세 튜닝의 어려운점**

|구분|어려움|영향|해결 난이도|
|---|---|---|---|
|1. 데이터 품질|양질의 레이블링 데이터 확보 어려움|모델 성능 저하|⭐⭐⭐⭐⭐|
|2. 과적합(Overfitting)|작은 데이터셋에서 과적합 발생|일반화 성능 저하|⭐⭐⭐⭐|
|3. 계산 자원|GPU 메모리, 학습 시간, 비용 문제|실험 제한|⭐⭐⭐⭐|
|4. 하이퍼파라미터 튜닝|최적의 learning rate, epoch 찾기 어려움|성능 편차 큼|⭐⭐⭐⭐|
|5. 재앙적 망각|사전학습 지식 손실|다른 태스크 성능 저하|⭐⭐⭐|
|6. 클래스 불균형|데이터 분포 편향|소수 클래스 예측 실패|⭐⭐⭐|


1. **데이터 품질 문제**
    - 전문가 레이블링 비용이 매우 높음
    - 레이블 오류가 모델에 직접 전파
    - 도메인 특화 데이터 수집의 어려움

2. **과적합 위험**
    - 사전학습 모델은 파라미터가 매우 많음 (BERT: 110M개)
    - 작은 학습 데이터로는 특정 패턴만 암기
    - validation loss와 training loss의 격차 발생

3. **계산 자원 제약**
    - 대형 모델은 최소 16GB GPU 필요
    - 배치 크기 제한으로 학습 불안정
    - 실험 반복 시간과 비용 증가

4. **하이퍼파라미터 민감성**
    - Learning rate가 너무 높으면 → 재앙적 망각
    - Learning rate가 너무 낮으면 → 학습 안 됨
    - 최적값이 태스크마다 다름

5. **재앙적 망각** (Catastrophic Forgetting)
    - 새 태스크 학습 시 이전 지식 손실
    - 사전학습된 언어 이해 능력 저하
    - 다중 태스크 동시 학습의 어려움

6. **클래스 불균형**
    - 실제 데이터는 불균형한 경우가 많음
    - 다수 클래스에 편향된 예측
    - 소수 클래스의 중요도가 높은 경우 문제



---



### 필요한 태스크 데이터셋 : NLI, STS (한국어)
- 모델 선정: **klue** 벤치마크의 **nli, sts** 태스크
- klue: https://huggingface.co/datasets/klue/klue
- nli viewer: https://huggingface.co/datasets/klue/klue/viewer/nli
- sts viewer: https://huggingface.co/datasets/klue/klue/viewer/sts

### 데이터셋을 수정했으므로 현재 데이터셋에 맞게 샘플링, feature selection 과정 코드 수정 필요



**all-MiniLM-L6-v2**:
- **범용** 임베딩 모델
- NLI, STS, 패러프레이즈 등 다양한 태스크로 훈련
- 추가 학습에 견고함 (Robust)

**sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2**:
- **다국어 범용** 임베딩 모델 (100+ 언어 지원)
- 패러프레이즈(의역) 탐지에 특화되어 훈련
- 다양한 언어의 STS, 패러프레이즈 태스크로 학습
- 추가 학습에 견고함 (다국어 안정성)

#### **범용성 높은 모델**은 **NLI** 학습으로 **STS** 능력이 **보강**되지만, **이미 특정 task (STS 등)에 특화된 모델**은 추가 학습이 오히려 **기존 특화 능력을 손상**시킴

  ### 모델 정보 확인

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModel

def quick_model_check(model_name):
    """핵심 정보만 빠르게 확인"""
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    st_model = SentenceTransformer(model_name, trust_remote_code=True)

    params = sum(p.numel() for p in model.parameters())

    print(f"Model: {model_name}")
    print(f"Parameters: {params:,}")
    print(f"Size: {params * 4 / 1024**2:.1f} MB")
    print(f"Embedding dim: {st_model.get_sentence_embedding_dimension()}")
    print(f"Max seq length: {st_model.max_seq_length}")
    print("-" * 40)

In [ ]:
quick_model_check("sentence-transformers/all-MiniLM-L6-v2")

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model: sentence-transformers/all-MiniLM-L6-v2
Parameters: 22,713,216
Size: 86.6 MB
Embedding dim: 384
Max seq length: 256
----------------------------------------


In [ ]:
quick_model_check('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Parameters: 117,653,760
Size: 448.8 MB
Embedding dim: 384
Max seq length: 128
----------------------------------------


In [ ]:
quick_model_check("jhgan/ko-sroberta-multitask")

Model: jhgan/ko-sroberta-multitask
Parameters: 110,618,112
Size: 422.0 MB
Embedding dim: 768
Max seq length: 128
----------------------------------------


In [ ]:
quick_model_check('klue/roberta-base')

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Model: klue/roberta-base
Parameters: 110,618,112
Size: 422.0 MB
Embedding dim: 768
Max seq length: 512
----------------------------------------


In [ ]:
quick_model_check('intfloat/multilingual-e5-large-instruct')

Model: intfloat/multilingual-e5-large-instruct
Parameters: 559,890,432
Size: 2135.8 MB
Embedding dim: 1024
Max seq length: 512
----------------------------------------


In [ ]:
quick_model_check('Alibaba-NLP/gte-multilingual-base')

modeling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model: Alibaba-NLP/gte-multilingual-base
Parameters: 305,368,320
Size: 1164.9 MB
Embedding dim: 768
Max seq length: 8192
----------------------------------------


## **임베딩 모델 미세 튜닝하기**

In [ ]:
# 1) 데이터 준비
from datasets import load_dataset

# 한국어 자연어이해 평가 벤치마크인 KLUE에서 nli을 로드
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("klue", "nli", split="train[:20000]")

# mnli에서 idx column을 제거했듯이, nli 데이터셋에 있는 guid column 제거
train_dataset = train_dataset.remove_columns("guid")
# 최종 데이터셋 확인
print(f"✅ KLUE NLI 훈련 데이터 로드 완료: {len(train_dataset):,}개")
print(f"컬럼: {train_dataset.column_names}")
print(f"\n샘플 데이터:")
for i in range(5):
  print(f'- {i + 1}번째 데이터: {train_dataset[i]}')

README.md: 0.00B [00:00, ?B/s]

nli/train-00000-of-00001.parquet:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

nli/validation-00000-of-00001.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

✅ KLUE NLI 훈련 데이터 로드 완료: 20,000개
컬럼: ['source', 'premise', 'hypothesis', 'label']

샘플 데이터:
- 1번째 데이터: {'source': 'NSMC', 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다', 'hypothesis': '힛걸 진심 최고로 멋지다.', 'label': 0}
- 2번째 데이터: {'source': 'NSMC', 'premise': '100분간 잘껄 그래도 소닉붐땜에 2점준다', 'hypothesis': '100분간 잤다.', 'label': 2}
- 3번째 데이터: {'source': 'NSMC', 'premise': '100분간 잘껄 그래도 소닉붐땜에 2점준다', 'hypothesis': '소닉붐이 정말 멋있었다.', 'label': 1}
- 4번째 데이터: {'source': 'NSMC', 'premise': '100분간 잘껄 그래도 소닉붐땜에 2점준다', 'hypothesis': '100분간 자는게 더 나았을 것 같다.', 'label': 1}
- 5번째 데이터: {'source': 'airbnb', 'premise': '101빌딩 근처에 나름 즐길거리가 많습니다.', 'hypothesis': '101빌딩 근처에서 즐길거리 찾기는 어렵습니다.', 'label': 2}


In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "sentence1": train_dataset["premise"],
        "sentence2": train_dataset["hypothesis"],
        "label": train_dataset["label"]
    }
)
df.head()

,sentence1,sentence2,label
0,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,0
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,2
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,1
3,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,1
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,2


In [ ]:
# df['label']별 빈도 수
df['label'].value_counts() # 0: 6851, 1: 6362, 2: 6787

,count
label,
0,6851
2,6787
1,6362


### **예제: 지도 학습 방법**

- **SFT: Supervised Fine-Tuning**
    - **SFT는 사전 학습된 모델을 레이블이 있는 데이터셋으로 추가 학습시켜 특정 작업에 맞게 조정하는 과정**
    - **"Supervised"의 의미**: 레이블된 데이터를 사용하여 학습 과정을 안내한다는 의미. 모델은 명시적인 피드백을 통해 **특정 입력에 대한 원하는 출력으로 매핑하는 방법을 학습**한다
    - 임베딩 모델 미세 튜닝에서는 SFT 대신 그냥 "Fine-tuning" 용어를 사용한다.(업계 관습?)

- **임베딩 모델을 미세 튜닝하는 가장 간단한 방법**
    - 이전에 했던 모델 훈련 과정을 그대로 따라하는 것
    - **모델 변경**: **bert-base-uncased** --> **sentence-transformers/all-MiniLM-L6-v2**(작고 속도가 좋다)


In [ ]:
# 1) 데이터 로드
from datasets import Dataset, load_dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import random
from tqdm import tqdm

# klue의 nli 서브셋을 로드
train_dataset = load_dataset("klue", "nli", split="train[:20000]")
# 이거 왜 빠졌지? 일부러 뺀건가??
train_dataset = train_dataset.filter(lambda x: x['label'] == 0)  # 수반만

# 삼중항 구성
train_examples = {"anchor": [], "positive": [], "negative": []}

# ✅ 리스트로 변환 후 shuffle
soft_negatives = list(train_dataset["hypothesis"])  # 리스트로 변환
random.shuffle(soft_negatives)

for row, soft_negative in zip(train_dataset, soft_negatives):
    train_examples["anchor"].append(row["premise"])
    train_examples["positive"].append(row["hypothesis"])
    train_examples["negative"].append(soft_negative)

train_dataset = Dataset.from_dict(train_examples)

# STSB를 위해 임베딩 유사도 평가자를 만듭니다.,
val_sts = load_dataset('klue', 'sts', split='validation')
print(val_sts)
print(val_sts['labels'])

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 519
})
Column([{'label': 4.9, 'real-label': 4.857142857142857, 'binary-label': 1}, {'label': 1.4, 'real-label': 1.428571428571429, 'binary-label': 0}, {'label': 1.3, 'real-label': 1.285714285714286, 'binary-label': 0}, {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}, {'label': 2.5, 'real-label': 2.5, 'binary-label': 0}])


In [ ]:
# 현재 코드가 실제로 작동하는지 확인
try:
    scores = [feature['labels']['label']/5.0 for feature in val_sts]
    print("작동함!\nscores 길이:", len(scores))
    print("첫 5개 점수:", scores[:5])
except Exception as e:
    print("에러 발생:", e)

작동함!
scores 길이: 519
첫 5개 점수: [0.9800000000000001, 0.27999999999999997, 0.26, 0.74, 0.5]


In [ ]:
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores= scores,  # KLUE는 'labels' 딕셔너리 안에 'label'이 있다.
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

print(f"✅ 평가 데이터 (KLUE STS): {len(val_sts):,}개")

print("\n✅ 모든 데이터 준비 완료! 이제 모델 훈련 가능합니다.")
print(f"\n데이터 요약:")
print(f"  - 훈련: KLUE NLI {len(train_dataset):,}개")
print(f"  - 평가: KLUE STS {len(val_sts):,}개")

✅ 평가 데이터 (KLUE STS): 519개

✅ 모든 데이터 준비 완료! 이제 모델 훈련 가능합니다.

데이터 요약:
  - 훈련: KLUE NLI 6,851개
  - 평가: KLUE STS 519개


In [ ]:
# 2) 모델 훈련 : 모델 정의 + 손실 함수 + 훈련 매개변수 + 모델 훈련
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 손실 함수 (MNR 사용)
train_loss = losses.MultipleNegativesRankingLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="finetuned_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.231200
200,0.096500


TrainOutput(global_step=215, training_loss=0.15616513407507607, metrics={'train_runtime': 31.243, 'train_samples_per_second': 219.281, 'train_steps_per_second': 6.882, 'total_flos': 0.0, 'train_loss': 0.15616513407507607, 'epoch': 1.0})

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.7708647446602384,
 'spearman_cosine': 0.7726404239290102,
 'pearson_euclidean': 0.7746953848805345,
 'spearman_euclidean': 0.7682156110798227,
 'pearson_manhattan': 0.7737119474701881,
 'spearman_manhattan': 0.7675019385118366,
 'pearson_dot': 0.7520425665233604,
 'spearman_dot': 0.7447094045378578,
 'pearson_max': 0.7746953848805345,
 'spearman_max': 0.7726404239290102}

In [ ]:
# 사전 훈련된 모델을 평가합니다.
original_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
evaluator(original_model)

{'pearson_cosine': 0.6643427782474073,
 'spearman_cosine': 0.6589523623849758,
 'pearson_euclidean': 0.646256473993339,
 'spearman_euclidean': 0.6417309818469817,
 'pearson_manhattan': 0.6482247434004565,
 'spearman_manhattan': 0.6441548929177391,
 'pearson_dot': 0.4151214234941033,
 'spearman_dot': 0.40698769732897894,
 'pearson_max': 0.6643427782474073,
 'spearman_max': 0.6589523623849758}

⚠️ **VRAM 비우기**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()



---



### **예제: 증식 SBERT**

- **임베딩 모델을 훈련하거나 미세 튜닝하는데 어려운 점**
    - 많은 양의 훈련 데이터가 필요하다는 점
    - 많은 모델이 10억 개 이상의 문장 쌍에서 훈련됨
    
- **증식(Argumented) SBERT** : https://arxiv.org/abs/2010.08240
    - **레이블이 있는 데이터가 적을 때, 문장 임베딩 모델을 미세 튜닝할 수 있도록 데이터를 증식하는 방법**
    - BERT vs. SBERT
        - **BERT** : 두 문장을 함께 입력받아 관계 예측 (**Cross-Encoder**)
        - **SBERT** : 각 문장을 독립적으로 임베딩하여 유사도 계산 (**Bi-Encoder**)
    - 참고: https://wikidocs.net/156176
- **증식(Argumented) SBERT의 과정(4단계 파이프라인)**
    1. **Gold 데이터셋으로 크로스 인코더 학습**
        - 정답이 달린 소량의 데이터(Gold 데이터셋)를 사용해서, 두 문장 관계(같은 의미/다름 등)를 잘 판단하는 **크로스 인코더**(BERT)를 미세조정
        - 비유: 국어 선생님에게 ‘참/거짓 문제’를 1만 개 가르쳐서, 문장 의미 판단을 잘하도록 훈련하는 단계
    2. **새로운 문장쌍 생성**
        - Gold 외의 문장을 이용해 다양한 문장쌍을 무작위로 조합
        - 비유: 선생님이 배운 내용 바탕으로 비슷한 문제를 스스로 만들어보기
    3. **크로스 인코더로 레이블 부여 (Silver 데이터셋**, 레이블은 있지만 정답이 아닐 수 있다)
        - 새 문장쌍마다 “이건 의미가 비슷/다름”을 크로스 인코더가 예측해서 점수 부여 → silver dataset 완성
        - 비유: 선생님이 새 문제에 답을 달아줌 (하지만 완벽히 정답은 아닐 수도 있음)
    4. **Gold + Silver로 바이 인코더(SBERT) 학습**
        - 이렇게 확장된 대량 데이터로 SBERT(문장 임베딩용 모델) 학습 → 빠르고 효율적
        - 비유: 학생이 ‘선생님 답안’을 보고 대량 연습을 통해 유사도 감각을 체득


![출처: 핸즈온 LLM](https://drive.google.com/uc?export=view&id=15C_AsJyz7YE2phpatFpnhuso8YJAyxGn)

- ⚠️ wandb 완전 비활성화
    - wandb : 모델 훈련 과정을 클라우드 대시보드에서 실시간으로 모니터링하게 해주는 서비스
    - Colab에는 보통 wandb가 미리 설치되어 있고, sentence-transformers(또는 내부에서 쓰는 transformers/Trainer)가 wandb가 설치돼 있으면 자동으로 로깅을 켜려 시도함
    - 비활성화 시킴

In [ ]:
# ✅ 이 셀을 "항상" 최상단(모델/라이브러리 import 전에) 실행하세요.
import os
os.environ["WANDB_DISABLED"] = "true"   # wandb 완전 비활성화
os.environ["WANDB_MODE"] = "disabled"   # (중복 안전장치)
os.environ["WANDB_SILENT"] = "true"    # 로그도 조용히

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

**단계 1:** 크로스 인코더를 미세 튜닝한다.(골드 데이터셋 만듦)


In [ ]:
# 정확한 크기 확인
from datasets import load_dataset, Dataset
klue_nli = load_dataset("klue", "nli", split="train")
print(f"전체 KLUE NLI 크기: {len(klue_nli)}")

전체 KLUE NLI 크기: 24998


총 24998개 중
골드 데이터셋은 7000개로 / 실버 17998개

In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import InputExample
from sentence_transformers.datasets import NoDuplicatesDataLoader
from sklearn.utils import resample

# 크로스 인코더를 위한 데이터셋 로드
dataset = load_dataset("klue", "nli", split="train[:5000]")
# (중립/모순)=0, (수반)=1
mapping = {2: 0, 1: 0, 0: 1}

# 먼저 pandas DataFrame 생성
gold = pd.DataFrame(
    {
        'sentence1': list(dataset['premise']),
        'sentence2': list(dataset['hypothesis']),
        'label': [mapping[label] for label in dataset['label']]
    }
)

print("원본 Gold 라벨 분포:")
print(gold['label'].value_counts())



# InputExample로 변환
gold_examples = [
    InputExample(texts=[row["sentence1"], row["sentence2"]], label=row["label"])
    for _, row in tqdm(gold.iterrows(), desc="Creating examples")
]

# 중복 제거 데이터로더
gold_dataloader = NoDuplicatesDataLoader(gold_examples, batch_size=32)

# 최종 gold는 균형 맞춘 버전 사용
gold = balanced_gold

원본 Gold 라벨 분포:
label
0    3293
1    1707
Name: count, dtype: int64


Creating examples: 5000it [00:00, 22425.16it/s]


In [ ]:


print(gold['label'].value_counts())

label
0    4611
1    2389
Name: count, dtype: int64


In [ ]:
print(f'gold_dataloader 개수: {len(gold_dataloader)}')
gold.head()

gold_dataloader 개수: 218


,sentence1,sentence2,label
0,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,1
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,0
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,0
3,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,0
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,0


In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# 골드 데이터셋에서 크로스 인코더를 훈련합니다.
cross_encoder = CrossEncoder('klue/bert-base', num_labels=2)
# fit: 훈련한다는 의미
cross_encoder.fit(
    train_dataloader=gold_dataloader,
    epochs=1,
    show_progress_bar=True,
    warmup_steps=100,
    use_amp=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


**단계 2:** 새로운 문장 쌍을 만듭니다.(실버 데이터셋 만듦)

In [ ]:
# 크로스 인코더로 레이블을 예측할 실버 데이터셋을 만듭니다.
silver = load_dataset("klue", "nli", split="train[7000:]")
pairs = list(zip(silver['premise'], silver['hypothesis']))

**단계 3:** 미세 튜닝된 크로스 인코더로 새로운 문장 쌍(실버 데이터셋)에 레이블을 할당합니다.

In [ ]:
import numpy as np

# 미세 튜닝된 크로스 인코더를 사용해 문장 쌍에 레이블을 할당합니다.
output = cross_encoder.predict(pairs, apply_softmax=True,
                               show_progress_bar=True)

# Dataset을 먼저 리스트로 변환
silver_premise = list(silver["premise"])
silver_hypothesis = list(silver["hypothesis"])

silver = pd.DataFrame(
    {
        "sentence1": silver_premise,
        "sentence2": silver_hypothesis,
        "label": np.argmax(output, axis=1)
    }
)
silver.head(10)

# 이제 라벨 분포 확인 가능
print("Silver 라벨 분포:")
print(silver['label'].value_counts())

Batches:   0%|          | 0/563 [00:00<?, ?it/s]

Silver 라벨 분포:
label
0    11779
1     6219
Name: count, dtype: int64


**단계 4:** 확장된 데이터셋(골드 데이터셋 + 실버 데이터셋)으로 바이 인코더(SBERT)를 훈련합니다.

In [ ]:
# 골드 데이터셋과 실버 데이터셋을 합칩니다.
data = pd.concat([gold, silver], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)
print(len(train_dataset))

24996


In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('klue', 'sts', split='validation')
# KLUE는 'labels' 딕셔너리 안에 'real-label'이 있다.
scores = [feature['labels']['real-label']/5.0 for feature in val_sts]
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores= scores,
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [ ]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('klue/bert-base')

# 손실 함수
train_loss = losses.MultipleNegativesRankingLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="augmented_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.188000
200,0.060700
300,0.048000
400,0.043100
500,0.043600
600,0.041600
700,0.034200


TrainOutput(global_step=782, training_loss=0.06329746197556596, metrics={'train_runtime': 81.6582, 'train_samples_per_second': 306.105, 'train_steps_per_second': 9.576, 'total_flos': 0.0, 'train_loss': 0.06329746197556596, 'epoch': 1.0})

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

# 결론: 데이터의 28% 정도 사용

{'pearson_cosine': 0.6897881878529657,
 'spearman_cosine': 0.6709495350309945,
 'pearson_euclidean': 0.6745910596945055,
 'spearman_euclidean': 0.6544921974078346,
 'pearson_manhattan': 0.6741600610307564,
 'spearman_manhattan': 0.6531472012495358,
 'pearson_dot': 0.6395917379430968,
 'spearman_dot': 0.6169567430894297,
 'pearson_max': 0.6897881878529657,
 'spearman_max': 0.6709495350309945}

In [ ]:
trainer.accelerator.clear()

[]

**단계 5**: 실버 데이터셋을 사용하지 않고 평가합니다.

In [ ]:
# 골드 데이터셋만 사용합니다.
data = pd.concat([gold], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)

# 모델
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="gold_only_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.213000
200,0.177700


TrainOutput(global_step=219, training_loss=0.19285255588897288, metrics={'train_runtime': 23.5117, 'train_samples_per_second': 297.681, 'train_steps_per_second': 9.314, 'total_flos': 0.0, 'train_loss': 0.19285255588897288, 'epoch': 1.0})

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

# 결론: 골드 데이터셋만 사용하는 것보다는 실버 데이터셋을 함꼐 사용하는 것이 모델 성능에 좋다.??
# 골드 데이터셋의 0과 1의 개수를 같게 해주니까 성능이 아주 조금 더 오르긴함.

{'pearson_cosine': 0.6381237037763409,
 'spearman_cosine': 0.6486730093313599,
 'pearson_euclidean': 0.6254822455483973,
 'spearman_euclidean': 0.6307548652596384,
 'pearson_manhattan': 0.6261876925608648,
 'spearman_manhattan': 0.6326209647996233,
 'pearson_dot': 0.5041863282054628,
 'spearman_dot': 0.5203311700025581,
 'pearson_max': 0.6381237037763409,
 'spearman_max': 0.6486730093313599}

실버 데이터셋과 골드 데이터셋을 모두 사용했을 때와 비교하면 골드 데이터셋만 사용한 경우 모델의 성능이 감소합니다!

골드 데이터셋을 통해 학습된 실버 데이터셋이 잘 학습되어야 함

### 🇺🇸 영어 (GLUE 데이터)
기준 모델: bert-base-uncased

Gold 데이터: 10,000개

Silver 데이터: 40,000개 (라벨분포 71%:29%)

결과:
- Gold만: 0.66
- Gold+Silver: 0.70
- 개선도: +약 0.04점 ✅
---

### 🇰🇷 한국어 (KLUE 데이터)
기준 모델: paraphrase-multilingual-MiniLM-L12-v2

Gold 데이터: 3,414개 (균형맞춤)

Silver 데이터: 19,998개 (라벨분포 74%:26%)

결과:
- Gold만: 0.66
- Gold+Silver: 0.3
- 변화: -0.36점 ❌
---

#### **영어: 고품질 Silver 라벨 생성 → 성능 향상**
#### **한국어: 저품질 Silver 라벨 생성 → 노이즈로 작용**
---
한국어 Cross-Encoder 문제:

1차 시도: 모든 것을 0으로 예측 (19,998:0)

2차 시도: 너무 많이 1로 예측 (5,138: 14,860)

⚠️ **VRAM 비우기**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()



---



## **비지도 학습**

- **비지도 학습**(Unsupervised learning)
    - 사전에 정의된 레이블 없이 모델을 훈련하는 기법
    - 레이블이 없는 대량의 텍스트만으로 고품질의 문장 임베딩을 생성하는 것을 목표로함

- **대표적인 비지도 학습 방법**
    - **SimCSE**(Simple Contrastive Learning of Sentence Embeddings): https://arxiv.org/abs/2104.08821
        - 드롭아웃(dropout)을 최소한의 데이터 증강 기법으로 활용하여, 동일한 문장을 두 번 인코딩한 결과를 긍정 쌍(positive pair)으로 삼아 대조 학습(contrastive learning)을 수행하는 방식

    - **CT**(Contrastive Tension):
        - 한 문장을 다른 모델(e.g., 번역 모델)을 통해 의미가 동일한 새로운 문장으로 생성하여 긍정 쌍을 만들고, 이 긍정 쌍의 유사도는 높이는 동시에 배치 내 모든 다른 문장들과의 유사도는 낮추는 강력한 대조 학습 방식

    - **TSDAE**(Transformer-based Sequential Denoising Auto-Encoder): https://arxiv.org/abs/2104.06979
        - 문장의 일부 단어를 **삭제하거나 순서를 섞는 등 손상(corrupt)시킨 뒤**, 원본 문장을 복원하도록 학습하는 **노이즈 제거 오토인코더(Denoising AutoEncoder) 구조**를 사용함
        - 인코더는 손상된 문장으로부터 의미 있는 임베딩을 생성해야 하고, 디코더는 이 임베딩 정보만을 이용해 완벽한 원본 문장을 재구성해야 함.
        - 이 과정에서 인코더는 문장의 핵심 의미를 압축적으로 담아내는 능력을 학습하게 되며, 이렇게 학습된 인코더가 고품질의 문장 임베딩을 생성함

    - **GPL**(Generative Pseudo-Labeling): https://arxiv.org/abs/2112.07577
        - 비지도 데이터를 활용해 **대규모의 고품질 학습 데이터를 자동으로 생성**(Generative Pseudo-Labeling)하여 모델을 학습시키는 파이프라인
    - 등 다양한 방법 존재

<img src="https://drive.google.com/uc?export=view&id=1dDuPhEKv8O2o85TgSN_6Z5WsUfWYc34h" width="80%">

### **예제: TSDAE**
- TSDAE(Transformer-based Sequential Denoising Auto-Encoder)
- **레이블 데이터가 전혀 없다고 가정**하며 인공적으로 레이블을 만들 필요가 없다.
- **TSDAE는 입력 문장에 잡음을 추가해 단어를 일정 비율 제거한다는 아이디어에 기반함**

- **핵심 개념**: 문장 임베딩이 정확할수록 재구성 문장이 정확하다!
    - 잡음이 추가된 오염된 문장을 인코더와 풀링 층에 통과시켜서 문장 임베딩에 매핑함
    - 이 문장 임베딩을 사용해 디코더가 오염된 문장에서 인공적인 잡음이 없는 원본 문장을 재구성함
    - 마스킹 된특정 단어를 재구성하는 마스크드 언어 모델링과 매우 비슷하지만, 마스킹 된특정 단어를 재구성이 아니라 **전체 문장을 재구성한다.**


- **잡음제거 과정에 사용할 토크나이저 다운로드한다.**

In [ ]:
# 추가적인 토크나이저를 다운로드합니다.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

- **데이터에서 두 문장(전제,가설)을 합치고 비지도 학습을 흉내내기 위해 레이브을 제거한다.**

  > 사실상 nli 데이터셋의 성격이 사라짐. 그냥 전제 + 가설로된 문장이 나열된 데이터셋

In [ ]:
from tqdm import tqdm
from datasets import Dataset, load_dataset
from sentence_transformers.datasets import DenoisingAutoEncoderDataset

# 전제와 가설을 하나의 문장으로 연결합니다. (레이블 사용 안함)
nli = load_dataset("klue", "nli", split="train")
flat_sentences = list(nli["premise"]) + list(nli["hypothesis"])
print(len(flat_sentences))  # ?건

# 커스텀 노이즈 함수 추가
def korean_noise_fn(text):
    words = text.split()
    # 삭제 비율을 0.6에서 0.2로 대폭 감소
    num_words_to_delete = max(1, int(len(words) * 0.2))
    if len(words) <= num_words_to_delete:
        return text  # 너무 짧으면 원본 그대로

    indices_to_delete = random.sample(range(len(words)), num_words_to_delete)
    return ' '.join([word for i, word in enumerate(words) if i not in indices_to_delete])

# 입력 데이터에 잡음을 추가합니다.
damaged_data = DenoisingAutoEncoderDataset(flat_sentences, noise_fn=korean_noise_fn)  # 커스텀 노이즈 함수 적용
print(len(damaged_data))

# 데이터셋을 만듭니다.
train_dataset = {"damaged_sentence": [], "original_sentence": []}
for data in tqdm(damaged_data):
    train_dataset["damaged_sentence"].append(data.texts[0])
    train_dataset["original_sentence"].append(data.texts[1])
train_dataset = Dataset.from_dict(train_dataset)

49996
49996


100%|██████████| 49996/49996 [00:00<00:00, 204344.99it/s]


In [ ]:
train_dataset[0]

{'damaged_sentence': '가전제품을 구입한다', 'original_sentence': '리처드는 가전제품을 구입한다.'}

In [ ]:
df = pd.DataFrame(
    {
    'damaged_sentence': train_dataset['damaged_sentence'],
    'original_sentence': train_dataset['original_sentence']
    }
)
df.head()

,damaged_sentence,original_sentence
0,힛걸 진심 최고다 그 어떤 히어로보다,힛걸 진심 최고다 그 어떤 히어로보다 멋지다
1,100분간 그래도 소닉붐땜에 2점준다,100분간 잘껄 그래도 소닉붐땜에 2점준다
2,100분간 그래도 소닉붐땜에 2점준다,100분간 잘껄 그래도 소닉붐땜에 2점준다
3,100분간 잘껄 그래도 소닉붐땜에,100분간 잘껄 그래도 소닉붐땜에 2점준다
4,101빌딩 근처에 즐길거리가 많습니다.,101빌딩 근처에 나름 즐길거리가 많습니다.


- **평가자 만들기**(evaluator)

In [ ]:
# 평가자 만들기
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('klue', 'sts', split='validation')
scores = [feature['labels']['label']/5.0 for feature in val_sts]
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=scores,
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)
print("첫 5개 점수:", scores[:5])

첫 5개 점수: [0.9800000000000001, 0.27999999999999997, 0.26, 0.74, 0.5]


- **모델 지정**
    - 풀링 레이어에 ('cls') 토큰 추가: 훈련시 위치 정보를 잃지 않게 하기 위해
    - cls :classification토큰

In [ ]:
# 모델 지정 : 풀링 레이어에 ('cls') 토큰 추가: 훈련시 위치 정보를 잃지 않게 하기 위해
from sentence_transformers import models, SentenceTransformer

# 임베딩 모델을 만듭니다.
word_embedding_model = models.Transformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model]) #pooling_model을 하나 추가

# modules = [word_embedding_model, pooling_model]
#                ↓                      ↓
#         1단계: 토큰 임베딩      2단계: 문장 임베딩
#        각 토큰이 768 차원벡터       1개의 768차원 벡터

- **손실 함수 지정**
    - 원본 문장을 재구성하는 손실 함수(DenoisingAutoEncoderLoss) 사용
    - tie_encoder_decoder=True : 인코더의 임베딩 층과 디코더의 출력 층의 가중치를 별도로 훈련하지 않고 동일한 가중치를 공유함 --> 한 층의 가중치를 업데이트하면 다른 층에도 반영됨

In [ ]:
from sentence_transformers import losses

# 잡음제거 오토 인코더 손실
train_loss = losses.DenoisingAutoEncoderLoss(
    embedding_model, tie_encoder_decoder=True
)
train_loss.decoder = train_loss.decoder.to("cuda")

Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.laye

- **모델 훈련**

In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="tsdae_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,10.621900
200,7.706000
300,6.639400
400,6.261500
500,6.059900
600,5.886100
700,5.767200
800,5.648500
900,5.582900
1000,5.465100


TrainOutput(global_step=3125, training_loss=5.498149255371094, metrics={'train_runtime': 343.0283, 'train_samples_per_second': 145.749, 'train_steps_per_second': 9.11, 'total_flos': 0.0, 'train_loss': 5.498149255371094, 'epoch': 1.0})

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.1792474026500443,
 'spearman_cosine': 0.18199088345914766,
 'pearson_euclidean': 0.20389973535884187,
 'spearman_euclidean': 0.1826886298603535,
 'pearson_manhattan': 0.20176175372966643,
 'spearman_manhattan': 0.17991696170849814,
 'pearson_dot': 0.16652886987484,
 'spearman_dot': 0.1624724001972133,
 'pearson_max': 0.20389973535884187,
 'spearman_max': 0.1826886298603535}

남은 학습들
1. **한국어 nli, sts 데이터셋 바꿔보기**
2. 한국어 임베딩 모델 바꿔보기 (범용성 넓고 성능 좋은 모델들로)
3. TSDAE 이외의 다른 비지도학습 방법 적용하여 성능 평가해보기
4. 결과 원인 분석 계속 해보기
- 풀링 방식 비교: cls vs mean vs max pooling 성능 차이
- 데이터셋 교체 실험: KLUE NLI → 다른 한국어 데이터셋으로 변경
- 태스크 미스매치 검증: NLI 데이터 훈련 → STS 평가의 적합성 분석
- 한국어 특화 모델 실험: 다국어 모델 vs 한국어 전용 모델
- 다른 비지도 방법 비교: TSDAE vs SimCSE vs 원본 모델
- 하이퍼파라미터 튜닝: 노이즈 비율, 에포크 수, 배치 크기, 학습률 조정